In [1]:
!pip install numpy
!pip install boto3
import numpy as np
import utils
import boto3
import pickle

In [2]:
import os

BASE_IMAGE = 'docker.io/pytorch/pytorch:1.0-cuda10.0-cudnn7-runtime'
bucket_name = 'elyra'
S3_END_POINT = "http://minio-kubeflow.apps.openmaru.ocp482.com"
S3_ACCESS_ID = "minio"
S3_ACCESS_KEY = "minio123"

In [3]:
s3_endpoint_url = 'http://minio-kubeflow.apps.openmaru.ocp482.com'
s3_access_key = "minio"
s3_secret_key = "minio123"

s3 = boto3.client(service_name='s3',
              	aws_access_key_id = s3_access_key,
              	aws_secret_access_key = s3_secret_key,
              	endpoint_url=s3_endpoint_url)

s3.list_buckets()['Buckets']

[{'Name': 'elyra',
  'CreationDate': datetime.datetime(2021, 8, 15, 12, 37, 19, 55000, tzinfo=tzlocal())},
 {'Name': 'mlpipeline',
  'CreationDate': datetime.datetime(2021, 7, 29, 13, 47, 55, 26000, tzinfo=tzlocal())}]

In [4]:
def write_to_store(bucket, data, key):
    s3.put_object(Body=pickle.dumps(data),
                      Bucket=bucket,
                      Key=key)
    
def gen_hyperparam_grid() -> int:
    '''Generate a list of namedtuples
    of hyperparams to evaluate
    '''

    grid = []
    for num_hidden_layers in [1,2,3]:
        for num_nodes in [1,2,3]:
            for activation in ['relu', 'sigmoid']:
                grid.append({'num_hidden_layers': num_hidden_layers,
                             'num_nodes': num_nodes,
                             'activation': activation
                            })


    write_to_store(bucket_name, grid, 'hyperparam_grid')

    return len(grid)

In [5]:
retcode = gen_hyperparam_grid()
if retcode <= 0: raise ValueError("Step 1b failed")